In [40]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
import pandas as pd
import numpy as np
import time
import math
import csv
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  
from features import *
data_loc = '../data/onepunch/'
pd.set_option('display.max_columns', None)  
from IPython.display import display
import time
import sys
import bisect

def load_data():
    print("Loading Data... ")
    data = {}
    for file_name, column_names in file_column_mappings.items():
        nice_name = file_name_name_mapping.get(file_name, None)
        if nice_name:
            print("Loading %s"%(nice_name))
            file_name = data_loc+file_name
            data[nice_name] = pd.read_csv(file_name,
                                          sep='|',
                                          index_col=False, 
                                          names=column_names,
                                          low_memory=False
                                        )
    print("Data Loaded!")
    return data

raw_data_dic = load_data()
raw_data = raw_data_dic['puborders']

Loading Data... 
Loading puborders
Data Loaded!


In [ ]:
def readTask(loc = './collate/OrdersDataFrame.csv'):
    with open(loc, 'r') as csvfile:
        raw_queue = pd.read_csv(csvfile, sep=',')
    raw_queue.drop(['time.1','minute'], axis=1, inplace=True)
    raw_queue["time"] = pd.to_datetime(raw_queue["time"], utc=True, errors='coerce')
    raw_queue.set_index("time",inplace=True)
    raw_queue.sort_index(ascending=True, inplace=True)
    return raw_queue

taskData = readTask()
display(taskData.head(2))
nameTask = list(taskData)

In [ ]:
def merge_task_and_order(raw_data,taskData):
    
    selected_data = raw_data.copy() # first add new columns
    
    print("before dropping empty time {}".format(raw_data.shape[0]))
    selected_data = selected_data.dropna(subset=['order_created_at'])
    total_num = selected_data.shape[0]
    print("after dropping empty time {}".format(total_num))
    
    selected_data["order_created_at"] = pd.to_datetime(raw_data["order_created_at"], utc=True, errors='coerce')
    selected_data.set_index("order_created_at",inplace=True)
    
    selected_data = selected_data[~selected_data.index.duplicated(keep='first')]
    # select and reindex task data to order time
    taskData = taskData.reindex(selected_data.index, method='nearest') 
    selected_data = selected_data.join(taskData, how='outer')
    return selected_data


merged_data = merge_task_and_order(raw_data,taskData)
nameOrder = list(merged_data)
display(merged_data.head(5))

# that's it

In [105]:
testdata = raw_data.copy()
testdata["order_created_at"] = pd.to_datetime(testdata["order_created_at"], utc=True, errors='coerce')
testdata.set_index("order_created_at",inplace=True)
test = testdata.index[0]

before dropping empty time 62610
after dropping empty time 62583


,order_id,order_state,order_updated_at,original_order_id,order_complexity,order_image_score,order_image_score_name,priority,order_deliverable,order_most_recent_failure_reason,site_id,order_local,slayer_job_id,order_site_name,sneaky,order_skip_markups,number_of_stories,e_number_of_stories_name,complexity_attributes_total_roof_facets_above_4_sqft,complexity_attributes_total_roof_facets,complexity_attributes_roof_square_footage,complexity_attributes_walls_square_footage,complexity_attributes_is_mfr,map_attributes_source,slayer_attributes_allows_mfr,slayer_attributes_is_mfr,slayer_attributes_user_first_capture,poser_release_number,structure_type,structure_type_name,poser_solve_status,hover_now,first_qa_datetime,first_qa_date,models_reprocessed,order_needs_to_be_reprocessed_flag,special_request,original_roof,first_qa_for_state,last_qa_for_state,first_qa_email,first_qa_site_name,first_qa_majority_of_doors_within_95_percent_of_standard_sizes,first_qa_map_overlay_accurate,first_qa_scale_within5_percent,first_qa_should_we_unfail_the_order,qa_comments,orthotag_list,level_list,pitch_estimate,roof_estimate,labeler_roof_estimate,markup_verifier_roof_estimate,markup_verifier_10_percent,pitch_actual,scaling_method,visible_on_ortho_map,door_areal_scale_diff,doors_used_for_scaling,door_scale_deviation,orthotag_tree_pass,orthotag_poor_image_quality,orthotag_none,orthotag_full_occlusion,orthotag_not_on_map,orthotag_tree_occlusion,orthotag_tree_fail,orthotag_missing_geometry,orthotag_shadow,poser_solved_image_count,poser_solved,failed_complete_timeline,failed_complete_progression,most_recent_failure_datetime,completed_count,failed_count,failed_or_complete_name_first,failed_or_complete_name_second,failed_or_complete_name_third,failed_or_complete_name_fourth,failed_or_complete_name_fifth,failed_or_complete_datetime_first,failed_or_complete_datetime_second,failed_or_complete_datetime_third,failed_or_complete_datetime_fourth,failed_or_complete_datetime_fifth,number_of_images,average_image_score,image_count_deleted,image_count_0_sides,image_count_1_side,image_count_2_sides,image_count_unscored,first_completion,most_recent_completion,unfail_premium_order_flag,turn_around_time_total,turn_around_time_first_completion,total_resolution_time_total,total_resolution_time_first_completion,human_bpo_time_waiting_total,human_bpo_time_waiting_first_completion,turn_around_time_wasted,human_bpo_time_wasted,labeling_time_total,labeling_time_first_completion,markup_verifying_time_total,markup_verifying_time_first_completion,model_building_time_total,model_building_time_first_completion,modeling_time_total,modeling_time_first_completion,model_completing_time_total,model_completing_time_first_completion,web_segmenting_time_total,web_segmenting_time_first_completion,model_segmenting_time_total,model_segmenting_time_first_completion,verifying_time_total,verifying_time_first_completion,texturing_time_total,texturing_time_first_completion,qa_time_total,openings_marking_time_total,openings_marking_time_first_completion,vps_marking_time_total,vps_marking_time_first_completion,primitive_marking_time_total,primitive_marking_time_first_completion,building_processing,complete,correcting_labels,cv_processing,exporting_3d_formats,extracting_measurements,failed,failed_building_processing,failed_cv_processing,failed_extracting_measurements,failed_image_processing,failed_primitives_processing,failed_pro_measurements_processing,failed_zulu_processing,image_processing,labeling,markup_verifying,model_segmenting,modeling,new,primitives_processing,pro_measurements_processing,qa,texturing,verifying,waiting_cv_processing,waiting_image_processing,waiting_labeling,waiting_markup,waiting_markup_verifying,waiting_model_segmenting,waiting_modeling,waiting_primitives_processing,waiting_pro_measurements_processing,waiting_qa,waiting_texturing,waiting_verifying,waiting_web_segmenting,waiting_zulu_processing,web_segmenting,zulu_processing
order_created_at,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

In [40]:
list(set(nameOrder).intersection(nameTask))
list(set(nameTask).intersection(nameTask))
list(set(nameOrder).intersection(nameOrder))

['order_updated_at',
 'first_qa_email',
 'web_segmenting_time_first_completion',
 'first_qa_datetime',
 'image_count_2_sides',
 'markup_verifier_10_percent',
 'model_segmenting_time_first_completion',
 'pitch_actual',
 'orthotag_list',
 'image_count_0_sides',
 'order_needs_to_be_reprocessed_flag',
 'failed_or_complete_name_third',
 'turn_around_time_first_completion',
 'model_completing_time_total',
 'order_deliverable',
 'primitive_marking_time_total',
 'level_list',
 'orthotag_tree_pass',
 'poser_solved_image_count',
 'image_count_unscored',
 'first_qa_scale_within5_percent',
 'door_scale_deviation',
 'total_resolution_time_total',
 'most_recent_failure_datetime',
 'orthotag_tree_fail',
 'modeling_time_total',
 'slayer_job_id',
 'order_state',
 'turn_around_time_wasted',
 'order_most_recent_failure_reason',
 'failed_complete_timeline',
 'vps_marking_time_total',
 'modeling_time_first_completion',
 'labeling_time_first_completion',
 'structure_type_name',
 'pitch_estimate',
 'complexi

# old method!

In [3]:
def readTask(loc = './collate/OrdersDataFrame.csv'):
    with open(loc, 'r') as csvfile:
        raw_queue = pd.read_csv(csvfile, sep=',')
    raw_queue.drop(['time.1','minute'], axis=1, inplace=True)
    raw_queue["time"] = pd.to_datetime(raw_queue["time"], utc=True, errors='coerce')
    raw_queue.index = raw_queue["time"].apply(lambda x: time.mktime(x.timetuple()))
    raw_queue.sort_index(ascending=True, inplace=True)
    #display(raw_queue.head(2))
    return raw_queue

taskData = readTask()
taskData.drop(['time'], axis=1, inplace=True)

In [88]:
new_columns = list(taskqueue)
timeHistKeyList = list(taskqueue.index)
ind = bisect.bisect_left(timeHistKeyList, 1528624790)
print(timeHistKeyList[ind-1])
key = timeHistKeyList[ind-1]
#display(taskqueue.loc[key])
print(new_columns)

1528624620.0
['building_processing', 'complete', 'correcting_labels', 'cv_processing', 'exporting_3d_formats', 'extracting_measurements', 'failed', 'failed_building_processing', 'failed_cv_processing', 'failed_extracting_measurements', 'failed_image_processing', 'failed_primitives_processing', 'failed_pro_measurements_processing', 'failed_zulu_processing', 'image_processing', 'labeling', 'markup_verifying', 'model_segmenting', 'modeling', 'new', 'primitives_processing', 'pro_measurements_processing', 'qa', 'texturing', 'verifying', 'waiting_cv_processing', 'waiting_image_processing', 'waiting_labeling', 'waiting_markup', 'waiting_markup_verifying', 'waiting_model_segmenting', 'waiting_modeling', 'waiting_primitives_processing', 'waiting_pro_measurements_processing', 'waiting_qa', 'waiting_texturing', 'waiting_verifying', 'waiting_web_segmenting', 'waiting_zulu_processing', 'web_segmenting', 'zulu_processing']


Loading Data... 
Loading puborders
Data Loaded!


In [23]:
def add_task_column(raw_data,taskqueue):
    
    new_columns = list(taskqueue)
    timeHistKeyList = list(taskqueue.index)
    timeHistKeyListsize = len(timeHistKeyList)
    
    selected_data = raw_data # first add new columns
    for new_name in new_columns:
        selected_data[new_name] = np.nan
    
    print("before dropping empty time {}".format(selected_data.shape[0]))
    selected_data.dropna(subset=['order_created_at'],inplace=True)
    total_num = selected_data.shape[0]
    print("after dropping empty time {}".format(total_num))
    
    selected_data["order_created_at"] = pd.to_datetime(raw_data["order_created_at"], utc=True, errors='coerce')
    
    for index, row in selected_data.iterrows():
        
        onetime = row['order_created_at']
        if(pd.isnull(onetime)):
            continue
        timestamp = time.mktime(onetime.timetuple())
        ind = bisect.bisect_left(timeHistKeyList, timestamp) # match with the nearest timestamp
        key_small = timeHistKeyList[ind-1]
        if ind < timeHistKeyListsize:
            key_big = timeHistKeyList[ind]
            key = key_small if abs(timestamp - key_small) < abs(timestamp - key_big) else key_big
        else:
            key = key_big
        new_data = taskqueue.loc[key]
        for feed_column in new_columns:
            selected_data.loc[index, feed_column] = new_data[feed_column] # add the needed data into those places
            #row[feed_column] = new_data[feed_column] 
            
        if index%100 == 0:
            sys.stdout.write(
                "\r>> total processed: {}% {}-{}={}".format(100*index/(total_num+0.0),key,timestamp,timestamp-key))
            sys.stdout.flush()
            
    return selected_data

merged_data = add_task_column(raw_data,taskData)
display(merged_data.head(2))

before dropping empty time 62338
after dropping empty time 62338
>> total processed: 26.4686066284% 1529934600.0-1529934572.0=-28.00000

KeyboardInterrupt: 

In [26]:
display(taskData.iloc[25000:25010])

,building_processing,complete,correcting_labels,cv_processing,exporting_3d_formats,extracting_measurements,failed,failed_building_processing,failed_cv_processing,failed_extracting_measurements,failed_image_processing,failed_primitives_processing,failed_pro_measurements_processing,failed_zulu_processing,image_processing,labeling,markup_verifying,model_segmenting,modeling,new,primitives_processing,pro_measurements_processing,qa,texturing,verifying,waiting_cv_processing,waiting_image_processing,waiting_labeling,waiting_markup,waiting_markup_verifying,waiting_model_segmenting,waiting_modeling,waiting_primitives_processing,waiting_pro_measurements_processing,waiting_qa,waiting_texturing,waiting_verifying,waiting_web_segmenting,waiting_zulu_processing,web_segmenting,zulu_processing
time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1.527048e+09,0,44,0,0,1,1,18,0,0,0,0,0,0,1,0,3,0,11,76,0,0,1,0,0,6,1,0,3,9,0,3,63,0,0,1327,6,14,21,0,4,0
1.527048e+09,1,44,0,0,0,1,18,0,0,0,0,0,0,1,1,3,0,12,75,0,0,0,0,1,8,1,0,3,9,0,4,65,0,0,1330,4,12,20,0,3,0
1.527048e+09,0,44,0,0,0,0,18,0,0,0,0,0,0,1,0,2,0,11,73,0,0,1,0,1,9,1,0,6,9,0,7,67,0,0,1330,5,12,16,0,6,0
1.527048e+09,1,44,0,0,0,1,18,0,0,0,0,0,0,1,0,1,0,12,74,0,0,2,0,1,9,1,0,4,10,0,9,68,0,0,1330,3,12,14,0,4,0
1.527049e+09,0,44,0,0,0,5,18,0,0,0,0,0,0,1,0,2,0,12,70,0,0,2,0,1,7,1,0,7,10,0,12,66,0,0,1332,1,12,17,0,3,0
1.527049e+09,1,44,0,0,0,3,18,0,0,0,0,0,0,1,0,1,0,12,71,0,0,1,0,2,9,1,0,8,10,0,16,66,0,0,1334,3,8,14,0,1,0
1.527049e+09,0,44,0,0,0,2,18,0,0,0,0,0,0,1,0,0,0,14,70,0,0,2,0,1,10,1,0,8,10,0,14,67,0,0,1337,3,7,14,0,1,0
1.527049e+09,1,44,0,0,0,1,18,0,0,0,0,0,0,1,1,1,0,14,70,0,0,1,0,1,9,1,0,9,9,0,14,65,0,0,1339,3,11,14,0,0,0
1.527049e+09,0,44,0,0,0,4,18,0,0,0,0,0,0,1,0,3,0,13,71,0,0,0,0,1,9,1,0,6,8,0,14,67,0,0,1339,4,9,16,0,0,0


In [7]:
display(merged_data.iloc[10000:10010])

,order_id,order_state,order_created_at,order_updated_at,original_order_id,order_complexity,order_image_score,order_image_score_name,priority,order_deliverable,order_most_recent_failure_reason,site_id,order_local,slayer_job_id,order_site_name,sneaky,order_skip_markups,number_of_stories,e_number_of_stories_name,complexity_attributes_total_roof_facets_above_4_sqft,complexity_attributes_total_roof_facets,complexity_attributes_roof_square_footage,complexity_attributes_walls_square_footage,complexity_attributes_is_mfr,map_attributes_source,slayer_attributes_allows_mfr,slayer_attributes_is_mfr,slayer_attributes_user_first_capture,poser_release_number,structure_type,structure_type_name,poser_solve_status,hover_now,first_qa_datetime,first_qa_date,models_reprocessed,order_needs_to_be_reprocessed_flag,special_request,original_roof,first_qa_for_state,last_qa_for_state,first_qa_email,first_qa_site_name,first_qa_majority_of_doors_within_95_percent_of_standard_sizes,first_qa_map_overlay_accurate,first_qa_scale_within5_percent,first_qa_should_we_unfail_the_order,qa_comments,orthotag_list,level_list,pitch_estimate,roof_estimate,labeler_roof_estimate,markup_verifier_roof_estimate,markup_verifier_10_percent,pitch_actual,scaling_method,visible_on_ortho_map,door_areal_scale_diff,doors_used_for_scaling,door_scale_deviation,orthotag_tree_pass,orthotag_poor_image_quality,orthotag_none,orthotag_full_occlusion,orthotag_not_on_map,orthotag_tree_occlusion,orthotag_tree_fail,orthotag_missing_geometry,orthotag_shadow,poser_solved_image_count,poser_solved,failed_complete_timeline,failed_complete_progression,most_recent_failure_datetime,completed_count,failed_count,failed_or_complete_name_first,failed_or_complete_name_second,failed_or_complete_name_third,failed_or_complete_name_fourth,failed_or_complete_name_fifth,failed_or_complete_datetime_first,failed_or_complete_datetime_second,failed_or_complete_datetime_third,failed_or_complete_datetime_fourth,failed_or_complete_datetime_fifth,number_of_images,average_image_score,image_count_deleted,image_count_0_sides,image_count_1_side,image_count_2_sides,image_count_unscored,first_completion,most_recent_completion,unfail_premium_order_flag,turn_around_time_total,turn_around_time_first_completion,total_resolution_time_total,total_resolution_time_first_completion,human_bpo_time_waiting_total,human_bpo_time_waiting_first_completion,turn_around_time_wasted,human_bpo_time_wasted,labeling_time_total,labeling_time_first_completion,markup_verifying_time_total,markup_verifying_time_first_completion,model_building_time_total,model_building_time_first_completion,modeling_time_total,modeling_time_first_completion,model_completing_time_total,model_completing_time_first_completion,web_segmenting_time_total,web_segmenting_time_first_completion,model_segmenting_time_total,model_segmenting_time_first_completion,verifying_time_total,verifying_time_first_completion,texturing_time_total,texturing_time_first_completion,qa_time_total,openings_marking_time_total,openings_marking_time_first_completion,vps_marking_time_total,vps_marking_time_first_completion,primitive_marking_time_total,primitive_marking_time_first_completion,building_processing,complete,correcting_labels,cv_processing,exporting_3d_formats,extracting_measurements,failed,failed_building_processing,failed_cv_processing,failed_extracting_measurements,failed_image_processing,failed_primitives_processing,failed_pro_measurements_processing,failed_zulu_processing,image_processing,labeling,markup_verifying,model_segmenting,modeling,new,primitives_processing,pro_measurements_processing,qa,texturing,verifying,waiting_cv_processing,waiting_image_processing,waiting_labeling,waiting_markup,waiting_markup_verifying,waiting_model_segmenting,waiting_modeling,waiting_primitives_processing,waiting_pro_measurements_processing,waiting_qa,waiting_texturing,waiting_verifying,waiting_web_segmenting,waiting_zulu_processing,web_segmenting,zulu_processing
10007,774162,waiting_qa,2018-06-24 23:59:41.

In [89]:
    
selected_data["order_created_at"] = pd.to_datetime(raw_data["order_created_at"], utc=True, errors='coerce')
for idx, onetime in enumerate(selected_data.order_created_at):
    if idx < 10:
        #print (onetime,time.mktime(onetime.timetuple()))
        timestamp = time.mktime(onetime.timetuple())
        ind = bisect.bisect_left(timeHistKeyList, timestamp)
        key = timeHistKeyList[ind-1]
        new_data = taskqueue.loc[key]
        
display(selected_data.head(1))

(Timestamp('2018-06-10 01:59:50.666520+0000', tz='UTC'), 1528624790.0)
(Timestamp('2018-06-10 18:36:24.225381+0000', tz='UTC'), 1528684584.0)
(Timestamp('2018-06-10 20:26:40.467408+0000', tz='UTC'), 1528691200.0)
(Timestamp('2018-06-11 04:57:14.381157+0000', tz='UTC'), 1528721834.0)
(Timestamp('2018-06-11 10:27:03.158752+0000', tz='UTC'), 1528741623.0)
(Timestamp('2018-06-11 11:22:21.262783+0000', tz='UTC'), 1528744941.0)
(Timestamp('2018-06-11 13:50:10.784184+0000', tz='UTC'), 1528753810.0)
(Timestamp('2018-06-11 14:19:45.696039+0000', tz='UTC'), 1528755585.0)
(Timestamp('2018-06-11 15:46:25.692593+0000', tz='UTC'), 1528760785.0)
(Timestamp('2018-06-11 19:40:34.567596+0000', tz='UTC'), 1528774834.0)


,order_id,order_state,order_created_at,order_updated_at,original_order_id,order_complexity,order_image_score,order_image_score_name,priority,order_deliverable,order_most_recent_failure_reason,site_id,order_local,slayer_job_id,order_site_name,sneaky,order_skip_markups,number_of_stories,e_number_of_stories_name,complexity_attributes_total_roof_facets_above_4_sqft,complexity_attributes_total_roof_facets,complexity_attributes_roof_square_footage,complexity_attributes_walls_square_footage,complexity_attributes_is_mfr,map_attributes_source,slayer_attributes_allows_mfr,slayer_attributes_is_mfr,slayer_attributes_user_first_capture,poser_release_number,structure_type,structure_type_name,poser_solve_status,hover_now,first_qa_datetime,first_qa_date,models_reprocessed,order_needs_to_be_reprocessed_flag,special_request,original_roof,first_qa_for_state,last_qa_for_state,first_qa_email,first_qa_site_name,first_qa_majority_of_doors_within_95_percent_of_standard_sizes,first_qa_map_overlay_accurate,first_qa_scale_within5_percent,first_qa_should_we_unfail_the_order,qa_comments,orthotag_list,level_list,pitch_estimate,roof_estimate,labeler_roof_estimate,markup_verifier_roof_estimate,markup_verifier_10_percent,pitch_actual,scaling_method,visible_on_ortho_map,door_areal_scale_diff,doors_used_for_scaling,door_scale_deviation,orthotag_tree_pass,orthotag_poor_image_quality,orthotag_none,orthotag_full_occlusion,orthotag_not_on_map,orthotag_tree_occlusion,orthotag_tree_fail,orthotag_missing_geometry,orthotag_shadow,poser_solved_image_count,poser_solved,failed_complete_timeline,failed_complete_progression,most_recent_failure_datetime,completed_count,failed_count,failed_or_complete_name_first,failed_or_complete_name_second,failed_or_complete_name_third,failed_or_complete_name_fourth,failed_or_complete_name_fifth,failed_or_complete_datetime_first,failed_or_complete_datetime_second,failed_or_complete_datetime_third,failed_or_complete_datetime_fourth,failed_or_complete_datetime_fifth,number_of_images,average_image_score,image_count_deleted,image_count_0_sides,image_count_1_side,image_count_2_sides,image_count_unscored,first_completion,most_recent_completion,unfail_premium_order_flag,turn_around_time_total,turn_around_time_first_completion,total_resolution_time_total,total_resolution_time_first_completion,human_bpo_time_waiting_total,human_bpo_time_waiting_first_completion,turn_around_time_wasted,human_bpo_time_wasted,labeling_time_total,labeling_time_first_completion,markup_verifying_time_total,markup_verifying_time_first_completion,model_building_time_total,model_building_time_first_completion,modeling_time_total,modeling_time_first_completion,model_completing_time_total,model_completing_time_first_completion,web_segmenting_time_total,web_segmenting_time_first_completion,model_segmenting_time_total,model_segmenting_time_first_completion,verifying_time_total,verifying_time_first_completion,texturing_time_total,texturing_time_first_completion,qa_time_total,openings_marking_time_total,openings_marking_time_first_completion,vps_marking_time_total,vps_marking_time_first_completion,primitive_marking_time_total,primitive_marking_time_first_completion,building_processing,complete,correcting_labels,cv_processing,exporting_3d_formats,extracting_measurements,failed,failed_building_processing,failed_cv_processing,failed_extracting_measurements,failed_image_processing,failed_primitives_processing,failed_pro_measurements_processing,failed_zulu_processing,image_processing,labeling,markup_verifying,model_segmenting,modeling,new,primitives_processing,pro_measurements_processing,qa,texturing,verifying,waiting_cv_processing,waiting_image_processing,waiting_labeling,waiting_markup,waiting_markup_verifying,waiting_model_segmenting,waiting_modeling,waiting_primitives_processing,waiting_pro_measurements_processing,waiting_qa,waiting_texturing,waiting_verifying,waiting_web_segmenting,waiting_zulu_processing,web_segmenting,zulu_processing
0,745942,complete,2018-06-10 01:59:50.666520